In [1]:
import numpy as np
import numpy.linalg as LA
import math

def homogene(P):
    return [P[0], P[1], 1]

def afine(P):
    return [round(P[0] / P[2]), round(P[1] / P[2])]

# Izdvaja kolone matrice
def kolone(matrica):
    prva = []
    druga = []
    treca = []
    
    for i in range(3):
        prva.append(matrica[i][0])
        druga.append(matrica[i][1])
        treca.append(matrica[i][2])
        
    return prva, druga, treca


# Naivni algoritam za odredjivanje projektivnog preslikavanja
# P1 je matrica sa kolonama alfaA, betaB, gamaC
# P2 je matrica sa kolonama alfa'A', beta'B', gama'C'
# P = P2 * P1^-1
def naivni(A, B, C, D, As, Bs, Cs, Ds):
    m1 = np.array([A, B, C]).T
    m2 = np.array([As, Bs, Cs]).T
    
    D = np.array([D]).T
    Ds = np.array([Ds]).T
   
    # D = M * X - matricna metoda
    X = np.linalg.inv(m1).dot(D)
    Y = np.linalg.inv(m2).dot(Ds)
   
    prva, druga, treca = kolone(m1)
    P1 = np.array([X[0] * prva, X[1] * druga, X[2] * treca]).T
   
    prva, druga, treca = kolone(m2)
    P2 = np.array([Y[0] * prva, Y[1] * druga, Y[2] * treca]).T
   
    P = np.matmul(P2, LA.inv(P1))
    return P

In [2]:
# Matrica korespodencije
def matrica_2x9(o, s):
    return np.array([
    [0, 0, 0, -s[2]*o[0], -s[2]*o[1], -s[2]*o[2], s[1]*o[0], s[1]*o[1], s[1]*o[2]],
    [s[2]*o[0], s[2]*o[1], s[2]*o[2], 0, 0, 0, -s[0]*o[0], -s[0]*o[1], -s[0]*o[2]]
    ])

# DLT algoritam za odredjivanje matrice projektivnog preslikavanja
def dlt(originali, slike):
    if len(originali) < 4 or len(slike) < 4 or len(originali) != len(slike):
        print('Pogresan broj koordinata')
        return None
        
    n = len(originali)
    M = [[]]
    
    for i in range(n):
        m = matrica_2x9(originali[i], slike[i])
        
        if i == 0:
            M = m
        else:
            M = np.concatenate((M, m), axis = 0)

    # SVD dekompozicija
    _, _, Vt = LA.svd(M, full_matrices=True)
    
    # Potrebna je poslednja vrsta od V transponovano
    P = Vt[-1]
    return P.reshape(3, 3)

In [3]:
def teziste(tacke, n):
    C_x = 0
    C_y = 0
    
    for t in tacke:
        C_x += t[0] / t[2]
        C_y += t[1] / t[2]
        
    return [C_x / n * 1.0, C_y / n * 1.0]

def rastojanje(tacke, n, C_x, C_y):
    ras = 0
    
    for t in tacke:
        ras += math.sqrt((t[0] / t[2] - C_x) ** 2 + (t[1] / t[2] - C_y) ** 2)

    return ras / n * 1.0

# Treba da se translira teziste u koordinatni pocetak
# Translira se za vektor CO - matrica translacije G
# Skalira se tako da je prosecna udaljenost od koordinatnog pocetka
# koren iz 2 - matrica homotetije S
# Matrica normalizacije T = SG
def normalizacija(tacke):
    n = len(tacke)
    [C_x, C_y] = teziste(tacke, n)

    ras = rastojanje(tacke, n, C_x, C_y)

    m_translacije = np.array([
        [1, 0, -C_x], 
        [0, 1, -C_y], 
        [0, 0, 1]
    ])
    
    m_homotetije = np.array([
        [math.sqrt(2) / ras * 1.0, 0, 0], 
        [0, math.sqrt(2) / ras * 1.0, 0], 
        [0, 0, 1]
    ])

    T = np.matmul(m_homotetije, m_translacije)

    nove_tacke = []
    for t in tacke:
        [x, y, z] = np.dot(T, [t[0], t[1], t[2]])
        nove_tacke.append([x, y, z])
    
    return nove_tacke, T

# DLT algoritam sa normalizacijom za odredjivanje matrice projektivnog preslikavanja
def dlt_normalizacija(originali, slike):
    if len(originali) < 4 or len(slike) < 4 or len(originali) != len(slike):
        print('Pogresan broj koordinata')
        return None
    
    originali_norm, T1 = normalizacija(originali)
    slike_norm, T2 = normalizacija(slike)
    
    Pt = dlt(originali_norm, slike_norm)
    
    P = np.matmul(np.matmul(LA.inv(T2), Pt), T1)

    return P

In [4]:
def promena_koordinata(M, tacke):
    
    nove_tacke = []
    for t in tacke:
        [x, y, z] = np.dot(M, [t[0], t[1], t[2]])
        nove_tacke.append([x, y, z])
        
    return nove_tacke

In [5]:
if __name__ == "__main__":
    a = [-3, 2, 1]
    b = [-2, 5, 2]
    c = [1, 0, 3]
    d = [-7, 3, 1]
    e = [2, 1, 2]
    f = [-1, 2, 1]
    g = [1, 1, 1]
    
    ap = [11, -12, 7]
    bp = [25, -8, 9]
    cp = [15, 4, 17]
    dp = [14, -28, 10]
    ep = [13, 8, 9]
    fp = [11, -4, 5]
    gp = [8.02, 4, 4]
    
    
    p_naivni = naivni(a, b, c, d, ap, bp, cp, dp)
    
    orig = np.array([a, b, c, d, e, f, g])
    print('Tacke sa test primera:')
    print(orig)
    
    slike = np.array([ap, bp, cp, dp, ep, fp, gp])
    print('\nSlike tacaka sa test primera:')
    print(slike)
 
    print("\nMatrica preslikavanja dobijena naivnim algoritmom:")
    print(np.round(p_naivni, 5))
    
    p_dlt = dlt(orig, slike)
    p_dlt = np.round(p_dlt, decimals = 5)
    
    print("\nMatrica preslikavanja dobijena DLT algoritmom koja je zaokruzena na 5 decimala:")
    print(p_dlt)
    
    print("\nSkalirana matrica DLT algoritma:")
    p_dlt = p_dlt / p_dlt[0][1] * 3
    print(p_dlt)
    
    print("\nSkalirana matrica DLT algoritma zaokruzena na 2 decimale:")
    print(np.round(p_dlt, 2))

    norm = dlt_normalizacija(orig, slike)
    
    print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom:")
    print(norm)
    
    print("\nSkalirana matrica DLT algoritma sa normalizacijom:")
    norm = norm / norm[0][1] * 3
    print(norm)
    
    print('\nZakljucak: Ista transformacija je u pitanju.')
    
    print('\nTestiramo invarijantnost u odnosu na promenu koordinata')
    M = [
        [0, 1, 2],
        [-1, 0, 3],
        [0, 0, 1]
    ]
    
    norig = promena_koordinata(M, orig)
    nslike = promena_koordinata(M, slike)
    
    print('\n Matrica promena koordinata:')
    print(np.array(M))
    
    print('\nKoordinate originalnih tacaka nakon primene matrice:')
    print(np.array(norig))
    
    print('\nKoordinate slike tacaka nakon primene matrice:')
    print(np.array(nslike))

    m1 = dlt(norig, nslike)
    m1s = LA.inv(M)
    m1s = np.matmul(m1s, m1)
    m1s = np.matmul(m1s, M)
    
    print('\nMatrica preslikavanja u starom koordinatnom sistemu:\n', np.round(m1s, 3))
    mm = m1s / m1s[0][1] * 3
    print('\nReskalirana matrica:\n', np.round(mm, 10))
    print('\nMatrica pre promena koordinata:\n', np.round(p_dlt, 5))
    
    print('\nIspostavi se da se matrice malo razlikuju tj. da dlt algoritam nije invarijantan u odnosu na promenu koordinata.')
    
    m2 = dlt_normalizacija(norig, nslike)
    m2s = LA.inv(M)
    m2s = np.matmul(m2s, m2)
    
    # Matrica preslikavanja u starom koordinatnom sistemu
    m2s = np.matmul(m2s, M)
    
    print('\nMatrica preslikavanja u starom koordinatnom sistemu:\n', m2s)
    
    nm2 = m2s / m2s[0][1] * 3
    print('\nReskalirana matrica:\n', np.round(nm2, 5))
    print('\nMatrica pre promene koordinata:\n', np.round(norm, 5))
    
    print('\nMatrice se ne razlikuju.')
    print('\nModifikovani dlt algoritam jeste invarijantan u odnosu na promenu koordinata.\n')

Tacke sa test primera:
[[-3  2  1]
 [-2  5  2]
 [ 1  0  3]
 [-7  3  1]
 [ 2  1  2]
 [-1  2  1]
 [ 1  1  1]]

Slike tacaka sa test primera:
[[ 11.   -12.     7.  ]
 [ 25.    -8.     9.  ]
 [ 15.     4.    17.  ]
 [ 14.   -28.    10.  ]
 [ 13.     8.     9.  ]
 [ 11.    -4.     5.  ]
 [  8.02   4.     4.  ]]

Matrica preslikavanja dobijena naivnim algoritmom:
[[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]]

Matrica preslikavanja dobijena DLT algoritmom koja je zaokruzena na 5 decimala:
[[ 5.0000e-05  3.2001e-01  5.3293e-01]
 [ 4.2645e-01 -2.0000e-05 -3.0000e-05]
 [-1.0660e-01 -1.0652e-01  6.3954e-01]]

Skalirana matrica DLT algoritma:
[[ 4.68735352e-04  3.00000000e+00  4.99606262e+00]
 [ 3.99784382e+00 -1.87494141e-04 -2.81241211e-04]
 [-9.99343771e-01 -9.98593794e-01  5.99550014e+00]]

Skalirana matrica DLT algoritma zaokruzena na 2 decimale:
[[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]]

Matrica preslikavanja dobijena DLT algoritmom sa normalizacijom:
[[ 3.89785465e-05  2.34697334e-01  3

In [6]:
# Upisuje koordinate odabranih tacaka
def click_event(event, x, y, flags, param):

    if event == cv2.EVENT_LBUTTONDOWN:
        koord.append([x, y])
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, '(' + str(x) + ', ' + str(y) + ')', (x, y), font, 0.5, (255, 0, 0), 2)
        cv2.imshow('image', img)

In [7]:
import cv2

koord = []
    
# Koordinate slika odredjujemo na osnovu prosecne duzine vertikalnih i horizontalnih duzi
# gornja leva tacka ostaje ista, a ostale se racunaju tako da obrazuju pravougaonik 
# i da su udaljene od te tacke za prosecnu duzinu
def nadji_slike(originali):

    A = originali[0]
    B = originali[1]
    C = originali[2]
    D = originali[3]
    
    AB = math.sqrt((A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2)
    DC = math.sqrt((D[0] - C[0]) ** 2 + (D[1] - C[1]) ** 2)
    
    AD = math.sqrt((A[0] - D[0]) ** 2 + (A[1] - D[1]) ** 2)
    BC = math.sqrt((B[0] - C[0]) ** 2 + (B[1] - C[1]) ** 2)
    
    pros_sirina = (AB + DC) / 2.0
    pros_duzina = (AD + BC) / 2.0

    slike = []
    slike.append([A[0], A[1], 1])
    slike.append([A[0], A[1] + pros_sirina, 1])
    slike.append([A[0] + pros_duzina, A[1] + pros_sirina, 1])
    slike.append([A[0] + pros_duzina, A[1], 1])

    return slike

def sortiranje(originali):

    originali.sort()
    
    if originali[1][1] < originali[0][1]:
        tmp = originali[0]
        originali[0] = originali[1]
        originali[1] = tmp
        
    if originali[2][1] < originali[3][1]:
        tmp = originali[2]
        originali[2] = originali[3]
        originali[3] = tmp
        
    return originali


# Distorzija
print('Na kraju programa pritisnite bilo koje dugme na tastaturi da izadjete iz istog.')

img = cv2.imread("dali.jpg", 1)
img = cv2.resize(img, (800, 450))
cv2.imshow('image', img)
cv2.setMouseCallback('image', click_event)

while True:
    if cv2.waitKey(1) and len(koord) == 4:
        break

originali = []

for i in range(4):
    originali.append(homogene(koord[i]))

originali = sortiranje(originali)
slike = nadji_slike(originali)

matrica = dlt_normalizacija(originali, slike)
matrica = np.round(matrica, decimals=10)

img = cv2.imread("dali.jpg", 1)
img = cv2.resize(img,(800, 450))
dst = cv2.warpPerspective(img, matrica, (800, 450))

cv2.imwrite('distorzija.jpg', dst)
cv2.imshow('Otklonjena projektivna distorzija', dst)
cv2.waitKey(0)

cv2.destroyAllWindows()

Na kraju programa pritisnite bilo koje dugme na tastaturi da izadjete iz istog.


In [8]:
import cv2
import numpy as np

# Obrisati crne delove krajnje slike
def trim(frame):
    if not np.sum(frame[0]):
        return trim(frame[1:])

    if not np.sum(frame[-1]):
        return trim(frame[:-2])

    if not np.sum(frame[:,0]):
        return trim(frame[:,1:])

    if not np.sum(frame[:,-1]):
        return trim(frame[:,:-2])
    
    return frame

koord = []

# Panorama
img = cv2.imread("leva_1.jpeg")
img = cv2.resize(img, (800,600))
cv2.imshow('image', img)
cv2.setMouseCallback('image', click_event)

while True:
    cv2.imshow("image", img)
    if cv2.waitKey(1) and len(koord) == 6:
        break

img = cv2.imread("desna_1.jpeg")
img = cv2.resize(img, (800,600))
cv2.imshow('image', img)
cv2.setMouseCallback('image', click_event)

while True:
    cv2.imshow("image", img)
    if cv2.waitKey(1) and len(koord) == 12:
        break
        
cv2.destroyAllWindows()

originali = []
for i in range(6):
    originali.append(homogene(koord[i]))

slike = []
for i in range(6):
    koord[i + 6][0] += 800
    slike.append(homogene(koord[i + 6]))

M = dlt_normalizacija(originali, slike)
M = np.round(M, decimals=10)
M = np.float32(M)

img = cv2.imread("leva_1.jpeg")
img = cv2.resize(img, (800,600))

img2 = cv2.imread("desna_1.jpeg")
img2 = cv2.resize(img2, (800,600))
   
levi_deo = cv2.warpPerspective(img, M, (1600,600))

j = [[1, 0, 0], 
     [0, 1, 0], 
     [0, 0, 1]]
j = np.float32(j)

levi_deo = cv2.warpPerspective(levi_deo, j, (800,600))

panorama = cv2.hconcat([levi_deo, img2])
panorama = trim(panorama)
cv2.imwrite('panorama.jpeg', panorama)
cv2.imshow('Panorama', panorama)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [9]:
print("Test primer koji ste mi poslali\n")

y1 = [0, 1, 1]
y2 = [1, 1, 1]
y3 = [-3, -1, 1]
y4 = [-1, 2, 1]
y5 = [2, 0, 1]

y1p = [-1, 1, 1]
y2p = [5, 0, 1]
y3p = [1, -2, 1]
y4p = [2, 3, 1]
y5p = [4, 3, 1]

p_naivni = naivni(y1, y2, y3, y4, y1p, y2p, y3p, y4p)

orig = np.array([y1, y2, y3, y4, y5])
print('Tacke sa test primera yi:')
print(orig)

slike = np.array([y1p, y2p, y3p, y4p, y5p])
print('\nSlike tacaka sa test primera yip:')
print(slike)

print("\nTestiranje algoritama koriscenjem 4 tacke.")

print("\nMatrica preslikavanja dobijena naivnim algoritmom:")
print(p_naivni)
p_naivni = p_naivni / p_naivni[0][0]
print("\nMatrica preslikavanja dobijena naivnim algoritmom tako da je P[0][0] = 1:")
print(p_naivni)

p_dlt = dlt([y1, y2, y3, y4], [y1p, y2p, y3p, y4p])
print("\nMatrica preslikavanja dobijena DLT algoritmom:")
print(p_dlt)
p_dlt = p_dlt / p_dlt[0][0]
print("\nMatrica preslikavanja dobijena DLT algoritmom tako da je P[0][0] = 1:")
print(p_dlt)

norm = dlt_normalizacija([y1, y2, y3, y4], [y1p, y2p, y3p, y4p])

print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom:")
print(norm)
print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom tako da je P[0][0] = 1:")
norm = norm / norm[0][0]
print(norm)

print('\nZakljucak: Ista transformacija je u pitanju.')

print("\nTestiranje algoritama koriscenjem 5 tacaka.")


p_dlt = dlt(orig, slike)
print("\nMatrica preslikavanja dobijena DLT algoritmom:")
print(p_dlt)
p_dlt = p_dlt / p_dlt[0][0]
print("\nMatrica preslikavanja dobijena DLT algoritmom tako da je P[0][0] = 1:")
print(p_dlt)

norm = dlt_normalizacija(orig, slike)
print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom:")
print(norm)
print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom tako da je P[0][0] = 1:")
norm = norm / norm[0][0]
print(norm)

print("\nTestiranje modifikovanog DLP algoritma koriscenjem sledecih tacaka.")

yn1 = [-2, -1, 1]
yn2 = [-1, -2, 1]
yn3 = [-7, 0, 1]
yn4 = [-2, 1, 1]
yn5 = [-1, -4, 1]

yn1p = [3, -2, 1]
yn2p = [4, 4, 1]
yn3p = [6, 0, 1]
yn4p = [1, 1, 1]
yn5p = [1, 3, 1]

orig_n = np.array([yn1, yn2, yn3, yn4, yn5])
print('\nTacke sa test primera yni:')
print(orig_n)

slike_n = np.array([yn1p, yn2p, yn3p, yn4p, yn5p])
print('\nSlike tacaka sa test primera ynip:')
print(slike_n)

norm = dlt_normalizacija(orig_n, slike_n)
print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom:")
print(norm)
print("\nMatrica preslikavanja dobijena DLT algoritmom sa normalizacijom tako da je P[0][0] = 1:")
norm = norm / norm[0][0]
print(norm)

Test primer koji ste mi poslali

Tacke sa test primera yi:
[[ 0  1  1]
 [ 1  1  1]
 [-3 -1  1]
 [-1  2  1]
 [ 2  0  1]]

Slike tacaka sa test primera yip:
[[-1  1  1]
 [ 5  0  1]
 [ 1 -2  1]
 [ 2  3  1]
 [ 4  3  1]]

Testiranje algoritama koriscenjem 4 tacke.

Matrica preslikavanja dobijena naivnim algoritmom:
[[-1.34375  0.9375  -1.21875]
 [-0.28125  2.4375  -2.15625]
 [-0.60625  0.1125   0.16875]]

Matrica preslikavanja dobijena naivnim algoritmom tako da je P[0][0] = 1:
[[ 1.         -0.69767442  0.90697674]
 [ 0.20930233 -1.81395349  1.60465116]
 [ 0.45116279 -0.08372093 -0.1255814 ]]

Matrica preslikavanja dobijena DLT algoritmom:
[[ 0.34411586 -0.24008083  0.31210508]
 [ 0.07202425 -0.62421017  0.55218592]
 [ 0.15525227 -0.0288097  -0.04321455]]

Matrica preslikavanja dobijena DLT algoritmom tako da je P[0][0] = 1:
[[ 1.         -0.69767442  0.90697674]
 [ 0.20930233 -1.81395349  1.60465116]
 [ 0.45116279 -0.08372093 -0.1255814 ]]

Matrica preslikavanja dobijena DLT algoritmom sa